# Check Overlap of items

In [2]:
import math
import pandas as pd
import numpy as np
import csv
import pandasql as ps

In [3]:
train_data = pd.read_csv("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/train.csv")

In [4]:
test_data = pd.read_csv("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/test.csv")

In [49]:
items_meta = pd.read_csv("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/item_metadata.csv")

## Get unique items

In [4]:
train_cos = train_data.loc[train_data['action_type'] == 'clickout item']
cos_items_train = train_cos.reference.unique()

In [5]:
test_cos = test_data.loc[test_data['action_type'] == 'clickout item']
cos_items_test = test_cos.reference.unique()

In [38]:
common_items = set(cos_items_train).intersection(cos_items_test)

## Dimension of intersections

In [47]:
len(common_items)/len(cos_items_train)

0.2953997499188272

In [48]:
len(common_items)/len(cos_items_test)

0.8141427796235827

In [56]:
all_items = items_meta.item_id.unique()

check for duplicates in items metadata

In [57]:
all_items.shape

(927142,)

In [58]:
items_meta.shape

(927142, 2)

check ratio of clickout items (compared to all items) in train and test

In [61]:
len(cos_items_test)/len(all_items)

0.11329763941230146

In [62]:
len(cos_items_train)/len(all_items)

0.31225637496737285

assuming there are no different ones (all items are collected in `item_metadata`)

using only clickout here, for a more accurate analysis on this side should consider all items with any interaction

# Assing items to Path (see if is need)

Assign items to path and then recommend by number of impression in that path

To do **Change**:
* group by `session_id`
* group by `reference` with POI or similar until new search
* count indipendently impression number
* create table with `item_id, impression_count, path_id` (even only possible)
* in test assign `session_id + target clickout` to a `path_id` (even multiple)
* assing list of arranged items to every target clickout for such path

`change algorithm considering always the cheapest price and if similar the one on top and if similar the one with more impression -> then consider a formula that takes into account all these aspects`

`check before in train and then in test set`

`see how many times is the cheapest or most impressed or first`

`dont know if it is in order or not, check on train set if there is distribution to confirm this`

In [63]:
train_data[:1000]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
9,00RL8Z82B2Z1,aff3928535f48,1541037542,10,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


# Check order of items in list

In [6]:
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out


def explode(df_in, col_expl):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, col_expl] = df[col_expl].apply(string_to_array)

    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df[col_expl].str.len())
         for col in df.columns.drop(col_expl)}
    )

    df_out.loc[:, col_expl] = np.concatenate(df[col_expl].values)
    df_out.loc[:, col_expl] = df_out[col_expl].apply(int)

    return df_out

In [74]:
train_cos

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...
115,02SRUT1NQYH1,3599a6f709eab,1541063864,35,clickout item,2795374,FI,"Krakow, Poland",mobile,NaN,2795374|5582964|1088390|2781070|1258068|127196...,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...
122,03K8AXBL4BX2,ec139e10b9238,1541100652,7,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...
176,03P4VFKK12UO,325fafb5fa450,1541107500,54,clickout item,65685,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,1306936|56482|2842358|6881276|65685|63259|6539...,178|104|110|94|57|96|46|61|48|35|50|38|59|44|5...
177,03P4VFKK12UO,325fafb5fa450,1541107538,55,clickout item,1320460,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,1306936|56482|2842358|6881276|65685|63259|6539...,178|104|110|94|57|96|46|61|48|35|50|38|59|44|5...
180,0473FZ8UNXRS,bcc452f3350eb,1541062532,3,clickout item,3143258,AU,"Legian, Indonesia",desktop,NaN,1258184|3866722|8929970|2315702|116619|1511641...,51|43|69|49|62|50|55|42|87|46|43|114|194|50|19...
181,066TUPQWUEV5,7157899be2839,1541102679,1,clickout item,2552514,MX,"Mazamitla, Mexico",mobile,NaN,2349076|2552514|7159866|3898458|1846017|725025...,54|39|55|51|41|94|24|38|59|24|72|314|46|40|183...
184,06S61EKCW1JY,22b9deb2da8f7,1541075258,1,clickout item,110591,FR,"Alicante, Spain",tablet,NaN,6721|6724|40109|147227|80983|6719|40718|119405...,36|45|99|48|49|50|49|56|40|63|35|95|40|78|132|...


In [57]:
train_cos_explode = explode(train_cos,"impressions")

In [72]:
train_cos_explode[:1000]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,prices,impressions,POS,is_clicked
0,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,3400638,0,0
1,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1253714,1,0
2,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,3367857,2,0
3,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,5100540,3,0
4,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1088584,4,0
5,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,666916,5,0
6,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,54833,6,0
7,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,2922310,7,0
8,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,9711560,8,0
9,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,109038,9,1


In [59]:
train_cos_explode["POS"] = train_cos_explode.groupby(['session_id','user_id','reference','timestamp']).cumcount()


In [66]:
train_cos_explode['is_clicked'] = np.where(train_cos_explode['reference'] == train_cos_explode['impressions'], 1, 0)


In [64]:
train_cos_explode = train_cos_explode.astype(str)

In [67]:
train_cos_explode

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,prices,impressions,POS,is_clicked
0,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,3400638,0,0
1,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1253714,1,0
2,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,3367857,2,0
3,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,5100540,3,0
4,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1088584,4,0
5,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,666916,5,0
6,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,54833,6,0
7,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,2922310,7,0
8,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,9711560,8,0
9,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,nan,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,109038,9,1


In [70]:
train_cos_explode.groupby(['POS','is_clicked']).count()

user_id  session_id  timestamp     step  action_type  \
POS is_clicked                                                         
0   0           1070409     1070409    1070409  1070409      1070409   
    1            508834      508834     508834   508834       508834   
1   0           1406994     1406994    1406994  1406994      1406994   
    1            167824      167824     167824   167824       167824   
2   0           1450529     1450529    1450529  1450529      1450529   
    1            119009      119009     119009   119009       119009   
3   0           1468854     1468854    1468854  1468854      1468854   
    1             95020       95020      95020    95020        95020   
4   0           1476238     1476238    1476238  1476238      1476238   
    1             81412       81412      81412    81412        81412   
5   0           1485604     1485604    1485604  1485604      1485604   
    1             65267       65267      65267    65267        65267   
6   0           1486276     1486276    1486276  1486276      1486276   
    1             57327       57327      57327    57327        57327   
7   0           1485986     1485986    1485986  1485986      1485986   
    1             50084       50084      50084    50084        50084   
8   0           1482982     1482982    1482982  1482982      1482982   
    1             45307       45307      45307    45307        45307   
9   0           1478666     1478666    1478666  1478666      1478666   
    1             41015       41015      41015    41015        41015   
10  0           1473228     1473228    1473228  1473228      1473228   
    1             36836       36836      36836    36836        36836   
11  0           1454345     1454345    1454345  1454345      1454345   
    1             33976       33976      33976    33976        33976   
12  0           1436346     1436346    1436346  1436346      1436346   
    1             31110       31110      31110    31110        31110   
13  0           1419079     1419079    1419079  1419079      1419079   
    1             28487       28487      28487    28487        28487   
14  0           1402114     1402114    1402114  1402114      1402114   
    1             26412       26412      26412    26412        26412   
...                 ...         ...        ...      ...          ...   
270 0                 2           2          2        2            2   
271 0                 2           2          2        2            2   
272 0                 2           2          2        2            2   
273 0                 2           2          2        2            2   
274 0                 2           2          2        2            2   
275 1                 1           1          1        1            1   
276 0                 1           1          1        1            1   
277 0                 1           1          1        1            1   
278 0                 1           1          1        1            1   
279 0                 1           1          1        1            1   
280 0                 1           1          1        1            1   
281 0                 1           1          1        1            1   
282 0                 1           1          1        1            1   
283 0                 1           1          1        1            1   
284 0                 1           1          1        1            1   
285 0                 1           1          1        1            1   
286 0                 1           1          1        1            1   
287 0                 1           1          1        1            1   
288 0                 1           1          1        1            1   
289 0                 1           1          1        1            1   
290 0                 1           1          1        1            1   
291 0                 1           1          1        1            1   
292 0                 1           1          1        1 

In [69]:
train_cos_explode.groupby('POS').count()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,prices,impressions,is_clicked
POS,,,,,,,,,,,,,
0,1579243,1579243,1579243,1579243,1579243,1579243,1579243,1579243,1579243,1579243,1579243,1579243,1579243
1,1574818,1574818,1574818,1574818,1574818,1574818,1574818,1574818,1574818,1574818,1574818,1574818,1574818
2,1569538,1569538,1569538,1569538,1569538,1569538,1569538,1569538,1569538,1569538,1569538,1569538,1569538
3,1563874,1563874,1563874,1563874,1563874,1563874,1563874,1563874,1563874,1563874,1563874,1563874,1563874
4,1557650,1557650,1557650,1557650,1557650,1557650,1557650,1557650,1557650,1557650,1557650,1557650,1557650
5,1550871,1550871,1550871,1550871,1550871,1550871,1550871,1550871,1550871,1550871,1550871,1550871,1550871
6,1543603,1543603,1543603,1543603,1543603,1543603,1543603,1543603,1543603,1543603,1543603,1543603,1543603
7,1536070,1536070,1536070,1536070,1536070,1536070,1536070,1536070,1536070,1536070,1536070,1536070,1536070
8,1528289,1528289,1528289,1528289,1528289,1528289,1528289,1528289,1528289,1528289,1528289,1528289,1528289


In [68]:
#check1 = train_cos_explode['POS','is_clicked'].astype(int)
train_cos_explode["POS"] = pd.to_numeric(train_cos_explode["POS"])
train_cos_explode["is_clicked"] = pd.to_numeric(train_cos_explode["is_clicked"])


In [61]:
train_cos_explode.POS.unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [16]:
train_cos_explode[train_cos_explode["reference"] = train_cos_explode["impressions"]].groupby(["POS"]).count.reset_index(name="count")



SyntaxError: invalid syntax (<ipython-input-16-d8c733634fcf>, line 1)

In [5]:
train_data.columns

Index(['user_id', 'session_id', 'timestamp', 'step', 'action_type',
       'reference', 'platform', 'city', 'device', 'current_filters',
       'impressions', 'prices'],
      dtype='object')

In [ ]:
query = """SELECT * FROM train_cos_explode LIMIT 10"""

print(ps.sqldf(query, locals()))

In [101]:
test_data.loc[test_data['user_id']=='000324D9BBUC']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
3090773,000324D9BBUC,89643988fdbfb,1541593272,1,search for destination,"Budapest, Hungary",BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
3090774,000324D9BBUC,89643988fdbfb,1541593820,2,filter selection,Car Park,BG,"Budapest, Hungary",desktop,Car Park,NaN,NaN
3090775,000324D9BBUC,89643988fdbfb,1541593882,3,interaction item image,1000915,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
3090776,000324D9BBUC,89643988fdbfb,1541593882,4,interaction item image,1000915,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
3090777,000324D9BBUC,89643988fdbfb,1541593902,5,interaction item image,1000915,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
3090778,000324D9BBUC,89643988fdbfb,1541593902,6,interaction item image,1000915,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
3090779,000324D9BBUC,89643988fdbfb,1541593931,7,interaction item image,1241375,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
3090780,000324D9BBUC,89643988fdbfb,1541593941,8,interaction item image,1241375,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
3090781,000324D9BBUC,89643988fdbfb,1541593941,9,interaction item image,1241375,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
3090782,000324D9BBUC,89643988fdbfb,1541593942,10,clickout item,NaN,BG,"Budapest, Hungary",desktop,Car Park,923407|1729121|1050792|97171|353141|106315|218...,47|78|76|76|77|79|55|75|78|69|77|56|72|61|47|6...


# Algorithm Just Order

In [77]:
def get_submission_target(df):
    """Identify target rows with missing click outs."""

    mask = df["reference"].isnull() & (df["action_type"] == "clickout item")
    df_out = df[mask]

    return df_out

In [78]:
df_target = get_submission_target(test_data)

In [79]:
df_explode = explode(df_target, 'impressions')

In [75]:
def group_concat(df, gr_cols, col_concat):
    """Concatenate multiple rows into one."""

    df_out = (
        df
        .groupby(gr_cols)[col_concat]
        .apply(lambda x: ' '.join(x))
        .to_frame()
        .reset_index()
    )

    return df_out

In [43]:
GR_COLS = ["user_id", "session_id", "timestamp", "step"]

In [82]:
df_expl_clicks = df_explode[GR_COLS + ["impressions"]]

In [84]:
df_out1 = df_expl_clicks.assign(impressions=lambda x: x["impressions"].apply(str))

In [86]:
df_out = group_concat(df_out1, GR_COLS, 'impressions')
df_out.rename(columns={'impressions': 'item_recommendations'}, inplace=True)
df_out

,user_id,session_id,timestamp,step,item_recommendations
0,000324D9BBUC,89643988fdbfb,1541593942,10,923407 1729121 1050792 97171 353141 106315 218...
1,0004Q49X39PY,9de47d9a66494,1541641157,1,2213014 3184842 10213134 4504242 4486372 38120...
2,0004Q49X39PY,beea5c27030cb,1541561202,1,3812004 3505150 3202894 2292254 1984229 222789...
3,00071784XQ6B,9617600e1ba7c,1541630328,2,22721 3067559 16121 22727 22854 22819 22764 14...
4,0008BO33KUQ0,2d0e2102ee0dc,1541636411,6,507861 2176280 1669587 502066 1352530 4342348 ...
5,000GO9NY6P4M,55dbafdbb9bab,1541594662,2,157710 1287958 160577 1618677 7231396 319866 4...
6,000IRHJS2DL9,f6ffffd20d43d,1541605541,12,102540 33191 2285010 2547840 20144 2808050 448...
7,000JB0UNEH23,7df07dc9fe26e,1541618174,1,2184212 2756867 2909994 4063406 3874514 950324...
8,000OWRCYEHKT,53e84da5c2dad,1541706095,3,5416250 7795438 3926654 8989706 6797450 722521...
9,000VBY1D6BP8,033fddaaa99af,1541587306,4,32256 32230 5411980 32246 32233 32235 5592656 ...


In [87]:
df_out.shape

(253573, 5)

In [88]:
df_out.to_csv("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/order_alg.csv", index=False)

## Check submission

In [52]:
SUBMISSION_COLUMNS = {'user_id', 'session_id', 'timestamp',
                      'step', 'item_recommendations'}

In [53]:
def get_test_session_set(df_test):
    """Identify sessions for which predictions have to be made."""

    mask = df_test.reference.isnull() & (df_test.action_type == 'clickout item')
    test_session_set = set(df_test[mask]['session_id'])

    return test_session_set

In [54]:
def check_passed(check=True):
    """Print result of check."""

    if check:
        print('> check passed')
    else:
        print('x check failed')

In [55]:
def check_duplicates(df):
    """Check if there are duplicate sessions in the df."""

    check_dupl = (len(df['session_id']) == len(df['session_id'].unique()))

    return check_dupl

In [56]:
def check_columns(df):
    """Check if the submission has the correct column names."""

    check_cols = (set(df.columns) == SUBMISSION_COLUMNS)

    return check_cols

In [57]:
def check_sessions(df_subm, df_test):
    """Check if the submission contains the correct sessions."""

    set_test_sessions = get_test_session_set(df_test)
    set_subm_sessions = set(df_subm['session_id'])
    check_sess = (set_test_sessions == set_subm_sessions)

    return check_sess

In [95]:
df_subm = pd.read_csv('~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/order_alg.csv')
df_test = test_data

In [96]:
check_cols = check_columns(df_subm)
check_passed(check_cols)

> check passed


In [97]:
check_dupl = check_duplicates(df_subm)
check_passed(check_dupl)

> check passed


In [98]:
check_sess = check_sessions(df_subm, df_test)
check_passed(check_sess)

> check passed


# Price Assignation

## Explode the two columns together

In [66]:
train = train_data

In [67]:
train_exercise = train[:50000]
train_exercise

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
9,00RL8Z82B2Z1,aff3928535f48,1541037542,10,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [47]:
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out


def explode(df_in, col_expl):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, col_expl] = df[col_expl].apply(string_to_array)

    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df[col_expl].str.len())
         for col in df.columns.drop(col_expl)}
    )

    df_out.loc[:, col_expl] = np.concatenate(df[col_expl].values)
    df_out.loc[:, col_expl] = df_out[col_expl].apply(int)

    return df_out

In [52]:
#explode(train_exercise, 'impressions','prices')
train_exercise.loc[:, 'prices'] = train_exercise['prices'].apply(string_to_array)

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [54]:
train_exercise.loc[:, 'impressions'] = train_exercise['impressions'].apply(string_to_array)

In [55]:
train_exercise

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,[],[]
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,[],[]
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,[],[]
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,[],[]
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,[],[]
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,[],[]
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,[],[]
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,[],[]
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,[],[]
9,00RL8Z82B2Z1,aff3928535f48,1541037542,10,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,[],[]


In [56]:
#remove the column impression to df_out
df_out = pd.DataFrame(
        {col: np.repeat(train_exercise[col].values,
                        train_exercise['impressions'].str.len())
         for col in train_exercise.columns.drop(['impressions','prices'])}
    )


In [1]:
df_out[:100]

NameError: name 'df_out' is not defined

In [58]:
df_out.loc[:, 'impressions'] = np.concatenate(train_exercise['impressions'].values)

In [59]:
df_out.loc[:, 'prices'] = np.concatenate(train_exercise['prices'].values)

In [60]:
df_out

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638,95
1,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,1253714,66
2,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3367857,501
3,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,5100540,112
4,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,1088584,95
5,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,666916,100
6,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,54833,101
7,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,2922310,72
8,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,9711560,82
9,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,109038,56


In [61]:
df_out.loc[:, 'impressions'] = df_out['impressions'].apply(int)
df_out.loc[:, 'prices'] = df_out['prices'].apply(int)

In [62]:
df_out

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638,95
1,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,1253714,66
2,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3367857,501
3,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,5100540,112
4,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,1088584,95
5,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,666916,100
6,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,54833,101
7,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,2922310,72
8,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,9711560,82
9,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,109038,56


In [7]:
def explode_item_price(df_in, col_explode1, col_explode2):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, col_explode1] = df[col_explode1].apply(string_to_array)
    df.loc[:, col_explode2] = df[col_explode2].apply(string_to_array)    

    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df[col_explode1].str.len())
         for col in df.columns.drop([col_explode1,col_explode2])}
    )

    df_out.loc[:, col_explode1] = np.concatenate(df[col_explode1].values)
    df_out.loc[:, col_explode2] = np.concatenate(df[col_explode2].values)  
    
    df_out.loc[:, col_explode1] = df_out[col_explode1].apply(int)
    df_out.loc[:, col_explode2] = df_out[col_explode2].apply(int)

    return df_out

In [73]:
data_explode_exercise = explode_item_price(train_exercise,'impressions','prices')

In [74]:
data_explode_exercise

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638,95
1,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,1253714,66
2,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3367857,501
3,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,5100540,112
4,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,1088584,95
5,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,666916,100
6,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,54833,101
7,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,2922310,72
8,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,9711560,82
9,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,109038,56


# Lower price algorithm

Get submission target and since that reorder list based on lower price, if equal choose higher displayed position  

In [10]:
def get_submission_target(df):
    """Identify target rows with missing click outs."""

    mask = df["reference"].isnull() & (df["action_type"] == "clickout item")
    df_out = df[mask]

    return df_out

In [11]:
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out

In [48]:
def explode_item_price(df_in, col_explode1, col_explode2):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, col_explode1] = df[col_explode1].apply(string_to_array)
    df.loc[:, col_explode2] = df[col_explode2].apply(string_to_array)    

    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df[col_explode1].str.len())
         for col in df.columns.drop([col_explode1,col_explode2])}
    )

    df_out.loc[:, col_explode1] = np.concatenate(df[col_explode1].values)
    df_out.loc[:, col_explode2] = np.concatenate(df[col_explode2].values)  
    
    df_out.loc[:, col_explode1] = df_out[col_explode1].apply(int)
    df_out.loc[:, col_explode2] = df_out[col_explode2].apply(int)

    return df_out

In [49]:
def group_concat(df, gr_cols, col_concat):
    """Concatenate multiple rows into one."""

    df_out = (
        df
        .groupby(gr_cols)[col_concat]
        .apply(lambda x: ' '.join(x))
        .to_frame()
        .reset_index()
    )

    return df_out

In [33]:
df_target_price = get_submission_target(test_data)

In [34]:
df_explode_price = explode_item_price(df_target_price, 'impressions','prices')

In [35]:
df_explode_price

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2059240,70
1,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2033381,46
2,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,1724779,48
3,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,127131,76
4,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,399441,65
5,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,103357,65
6,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,127132,106
7,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,1167004,66
8,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,4491766,87
9,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2249874,43


In [36]:
df_explode_price["POS"] = df_explode_price.groupby(['user_id','session_id','timestamp','step']).cumcount()

In [40]:
df_expl_price = df_explode_price.groupby(['user_id','session_id','timestamp','step']).apply(lambda x: x.sort_values(['prices','POS'])).reset_index(drop=True)


In [44]:
df_expl_price = df_expl_price[GR_COLS + ["impressions"]]



In [46]:
df_out1 = df_expl_price.assign(impressions=lambda x: x["impressions"].apply(str))



In [50]:
df_out = group_concat(df_out1, GR_COLS, 'impressions')


In [51]:
df_out.rename(columns={'impressions': 'item_recommendations'}, inplace=True)
df_out

,user_id,session_id,timestamp,step,item_recommendations
0,000324D9BBUC,89643988fdbfb,1541593942,10,923407 4234718 147907 2182130 927351 2094950 1...
1,0004Q49X39PY,9de47d9a66494,1541641157,1,2292254 8153310 2712428 2227026 3202894 350515...
2,0004Q49X39PY,beea5c27030cb,1541561202,1,7101352 4476010 10024490 2292254 1736323 54973...
3,00071784XQ6B,9617600e1ba7c,1541630328,2,22854 22772 22819 3498754 150825 22713 1478189...
4,0008BO33KUQ0,2d0e2102ee0dc,1541636411,6,1669587 4272108 507861 2176280 1888653 1830637...
5,000GO9NY6P4M,55dbafdbb9bab,1541594662,2,1479743 319866 483691 4058060 9910744 160577 1...
6,000IRHJS2DL9,f6ffffd20d43d,1541605541,12,3054956 102540 20144 1652615 33191 20166 22944...
7,000JB0UNEH23,7df07dc9fe26e,1541618174,1,4063406 9503248 3884098 2756867 9670320 252608...
8,000OWRCYEHKT,53e84da5c2dad,1541706095,3,5416250 6797450 8989706 3926654 7225216 779543...
9,000VBY1D6BP8,033fddaaa99af,1541587306,4,32256 5411980 6865398 32246 5592656 32230 3223...


In [58]:
df_out.to_csv("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/price_alg.csv", index=False)

In [59]:
df_subm = pd.read_csv('~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/price_alg.csv')
df_test = test_data

In [60]:
check_cols = check_columns(df_subm)
check_passed(check_cols)



> check passed


In [61]:
check_dupl = check_duplicates(df_subm)
check_passed(check_dupl)


> check passed


In [62]:
check_sess = check_sessions(df_subm, df_test)
check_passed(check_sess)

> check passed
